In [50]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from pathlib import Path
import re

In [55]:
REP_DATA_PATH = Path("data/train_x.csv")
rep_data = pd.read_csv(REP_DATA_PATH, header=None).to_numpy()
rep_data = rep_data.reshape(1600, 128, 3)
rep_data = np.expand_dims(rep_data, axis=3)
print(rep_data.shape)

def representative_dataset():
    for value in range(100):
        _ = tf.expand_dims(rep_data[value], axis=0)
        _ = _.numpy()
        yield [_.astype(np.float32)]

MODEL_SOURCE = Path("models/ver0")
converter = tf.lite.TFLiteConverter.from_saved_model(str(MODEL_SOURCE))
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.int8  # or tf.uint8 TFLITE_BUILTINS_INT8
# converter.inference_output_type = tf.int8  # or tf.uint8
tflite_model = converter.convert()

tf_lite_path = Path("models") / (MODEL_SOURCE.name + ".tflite")
with open(tf_lite_path, "wb") as f:
    f.write(tflite_model)

(1600, 128, 3, 1)


2022-06-08 15:45:43.306082: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-06-08 15:45:43.306110: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-06-08 15:45:43.306234: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: models/ver0
2022-06-08 15:45:43.307596: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-06-08 15:45:43.307610: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: models/ver0
2022-06-08 15:45:43.312707: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-06-08 15:45:43.365185: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: models/ver0
2022-06-08 15:45:43.382529: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 76295 microseconds.
fu

In [56]:
# convert from tflite to C array
os.system(f"xxd -i {tf_lite_path} > models/micro_model.cpp")

0

In [57]:
# copy raw C array to embedded environment and add header

SRC_MODEL_PATH = Path("models/micro_model.cpp")
DEST_MODEL_PATH = Path("../main/src/micro_model.cpp")

def read_len() -> int:
    """read C-array length from file"""
    with open(SRC_MODEL_PATH) as f:
        for line in f:
            if line.startswith("unsigned int"):
                length = re.search("([0-9])+;", line)
                # take first match, remove semicolon, coerce
                length = int(length[0][:-1])
                return int(length)


def read_array() -> str:
    """read C-array from file"""
    array = ""
    with open(SRC_MODEL_PATH) as f:
        for i, line in enumerate(f, 1):
            if i == 1: continue # start
            if line.startswith("};"): break # end
            array += line
    return array


def get_cfile_header(array_length) -> str:
    return (f"""
// We need to keep the data array aligned on some architectures.
#include "micro_model.h"

// memory management
#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

const unsigned int micro_model_len = {array_length};
const unsigned char micro_model[] DATA_ALIGN_ATTRIBUTE = """ + "{\n"
)

def get_cfile_footer() -> str:
    return "};"

def compose_c_file() -> str:
    array_length = read_len()
    array = read_array()
    header = get_cfile_header(array_length)
    footer = get_cfile_footer()
    doc = header + array + footer
    return doc

def copy_to_mcu() -> None:
    doc = compose_c_file()
    with open(DEST_MODEL_PATH, "w") as f:
        f.write(doc)

copy_to_mcu()